In [56]:
setprecision(256)

using Plots

plotly()

C = 3000000 ^ 2

9000000000000

## Generator testów

In [69]:
function gen_test(satellites)
    # wektor [x; y; z; t] - (x, y, z): nasza pozycja, t: błąd zegara
    X = vcat(Array{BigFloat}(2000 * rand(3) - 1000) .^ 3, Array{BigFloat}(100 * randn(1)) .^ 3)
    data = Array{BigFloat, 2}(4, satellites)
    for i = 1:satellites
        v = Array{BigFloat}(2000 * rand(3) - 1000) .^ 3
        data[:, i] = X + vcat(v, norm(v) / sqrt(C))
    end
    return data, X
end

gen_test (generic function with 1 method)

## Dane do testów

In [71]:
data, solution₁ = gen_test(4)

# data[1, :] : wektor współrzędnych x satelit
# data[2, :] : wektor współrzędnych y satelit
# data[3, :] : wektor współrzędnych z satelit
# data[4, :] : wektor czasów zegara satelit

# data[:, i] : dane z i-tej satelity
# solution₁ = poszukiwane rozwiązanie (niekoniecznie jedyne)

# Funkcja F jak w sprawozdaniu
function f(x::Array{BigFloat})
    return ((-data .+ x) .^ 2)' * [1, 1, 1, -C*C]
end


JacobianArray = Array{Function, 2}(4, 4)
for i = 1:4
    for j = 1:3
        JacobianArray[i, j] = function(x) return 2*x[j] - 2*data[j, i] end
    end
    JacobianArray[i, 4] = function(x) return 2*C*data[4, i] - 2*C*x[4] end
end

# Jakobian jak w sprawozdaniu
function Jacobian(x::Array{BigFloat})
    return map((f) -> f(x), JacobianArray)
end


Jacobian (generic function with 1 method)

# Metoda Newtona

In [73]:
function prnt(a::Array{Float32})
    @printf "%.4e, %.4e, %.4e, %.4e" a[1] a[2] a[3] a[4]
end

function NewtonMethod(x::Array{BigFloat}, ϵ)
    while true 
        if norm(f(x)) < ϵ
            return x
        end
        @printf "x - sol: ["
        prnt(abs.(Array{Float32}(x - solution₁)))
        @printf "]\tf(x) : ["
        prnt(abs.(Array{Float32}(f(x))))
        @printf "]\n"
        δ = Jacobian(x) \ f(x)
        x = x - δ
    end
end

NewtonMethod(Array{BigFloat}([1, 1, 1, 0]), 1e-50)

x - sol: [5.6510e+08, 6.4568e+08, 4.2134e+08, 7.2620e+05]	f(x) : [4.7484e+24, 4.7475e+24, 4.7466e+24, 4.7488e+24]
x - sol: [8.5280e+11, 6.5808e+11, 1.4915e+11, 3.6308e+05]	f(x) : [4.1281e+21, 4.1281e+21, 4.1281e+21, 4.1281e+21]
x - sol: [4.1261e+11, 3.1840e+11, 7.2164e+10, 1.7567e+05]	f(x) : [1.0310e+21, 1.0310e+21, 1.0310e+21, 1.0310e+21]
x - sol: [1.9298e+11, 1.4891e+11, 3.3751e+10, 8.2161e+04]	f(x) : [2.5666e+20, 2.5666e+20, 2.5666e+20, 2.5666e+20]
x - sol: [8.4073e+10, 6.4876e+10, 1.4704e+10, 3.5794e+04]	f(x) : [6.3103e+19, 6.3103e+19, 6.3103e+19, 6.3103e+19]
x - sol: [3.1394e+10, 2.4226e+10, 5.4906e+09, 1.3366e+04]	f(x) : [1.4765e+19, 1.4765e+19, 1.4765e+19, 1.4765e+19]
x - sol: [8.2278e+09, 6.3491e+09, 1.4390e+09, 3.5030e+03]	f(x) : [2.8554e+18, 2.8554e+18, 2.8554e+18, 2.8554e+18]
x - sol: [9.2161e+08, 7.1118e+08, 1.6118e+08, 3.9238e+02]	f(x) : [2.8401e+17, 2.8401e+17, 2.8401e+17, 2.8401e+17]
x - sol: [1.4434e+07, 1.1139e+07, 2.5245e+06, 6.1455e+00]	f(x) : [4.3786e+15, 4.3786e+15

4-element Array{BigFloat,1}:
  5.651007276220236858539610356056223000222243280261724079072072614115300516734455e+08
 -6.456770172691195434356496647304739620844652297474075423137731921118388593907262e+08
 -4.213413900025584967266641945146085487787859334287050115381247983442186351829521e+08
  7.262042950408124809561456889923951382427112566437666322507959298067151606348724e+05

In [2]:
# Metoda algebraiczna